# Forward Solver Demo

This notebook demonstrates the forward solver for the Poisson equation with point sources.

## Problem
$$-\Delta u = \sum_i q_i \delta(x - x_i) \quad \text{in } \Omega$$
$$\frac{\partial u}{\partial n} = 0 \quad \text{on } \partial\Omega$$

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt

# Add src to path
import sys
sys.path.insert(0, '../src')

from forward_solver import (
    create_disk_mesh,
    solve_poisson_zero_neumann,
    get_boundary_values,
    plot_solution,
    plot_boundary_solution
)

from mesh_utils import get_mesh_info, plot_mesh

## 1. Create Mesh

In [ ]:
# Create a disk mesh
mesh, cell_tags, facet_tags = create_disk_mesh(radius=1.0, resolution=0.05)

# Print mesh info
info = get_mesh_info(mesh)
print("Mesh Information:")
for key, value in info.items():
    print(f"  {key}: {value}")

## 2. Define Source Locations

In [ ]:
# Define sources: ((x, y), intensity)
# Positive intensity = source
# Negative intensity = sink
# NOTE: Total intensity must sum to zero for pure Neumann problem!

sources = [
    ((-0.2, 0.45), 1.0),    # Positive
    ((0.6, 0.35), 1.0),     # Positive
    ((-0.6, -0.3), -1.0),   # Negative
    ((0.5, -0.1), -1.0),    # Negative
]

# Verify sum is zero
total = sum(q for _, q in sources)
print(f"Total intensity: {total} (should be 0)")

In [ ]:
# Plot mesh with source locations
plot_mesh(mesh, sources)

## 3. Solve Forward Problem

In [ ]:
# Solve the PDE
u = solve_poisson_zero_neumann(mesh, sources, polynomial_degree=1)

print(f"Solution computed!")
print(f"  Min: {u.x.array.min():.6f}")
print(f"  Max: {u.x.array.max():.6f}")
print(f"  Mean: {u.x.array.mean():.6f}")

In [ ]:
# Plot solution
plot_solution(u, sources)

## 4. Extract Boundary Measurements

These boundary values are what we would "measure" in the inverse problem.

In [ ]:
# Get solution values on the boundary
angles, boundary_values = get_boundary_values(u)

print(f"Extracted {len(angles)} boundary points")
print(f"Boundary value range: [{boundary_values.min():.6f}, {boundary_values.max():.6f}]")

In [ ]:
# Plot boundary values
plot_boundary_solution(angles, boundary_values)

## 5. Save Results for Inverse Problem

Save the boundary data that will be used as input for the inverse solver.

In [ ]:
# Save boundary data
boundary_data = {
    'angles': angles,
    'values': boundary_values,
    'sources_true': sources  # Ground truth for validation
}

np.savez('../results/boundary_measurements.npz', **boundary_data)
print("Saved to ../results/boundary_measurements.npz")

## 6. Experiment: Effect of Number of Sources

In [ ]:
# Try different source configurations
import random

def random_sources(n_positive, n_negative, radius=0.8):
    """Generate random balanced sources inside the disk."""
    sources = []
    
    for _ in range(n_positive):
        # Random point inside disk
        while True:
            x, y = random.uniform(-radius, radius), random.uniform(-radius, radius)
            if x**2 + y**2 < radius**2:
                break
        sources.append(((x, y), 1.0))
    
    for _ in range(n_negative):
        while True:
            x, y = random.uniform(-radius, radius), random.uniform(-radius, radius)
            if x**2 + y**2 < radius**2:
                break
        sources.append(((x, y), -1.0))
    
    return sources

# Generate 3 positive and 3 negative sources
random.seed(42)
sources_random = random_sources(3, 3)
print("Random sources:")
for (x, y), q in sources_random:
    print(f"  ({x:.3f}, {y:.3f}): {q:+.1f}")

In [ ]:
# Solve with random sources
u_random = solve_poisson_zero_neumann(mesh, sources_random)
plot_solution(u_random, sources_random)

## Next Steps: Inverse Problem

The inverse problem will:
1. Take the boundary measurements `boundary_values`
2. Reconstruct the source locations and intensities

Approaches to implement:
- Optimization-based (minimize ||u_measured - u_computed||²)
- MUSIC algorithm (spectral methods)
- Bayesian inference
- Neural network methods